In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import nltk
import sklearn
import matplotlib.pyplot as plt

In [2]:
seed_value = 123
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import *
from keras.regularizers import l1,l2, L1L2

In [4]:
# load in tweets
with open('data/pickled_tweets/home_2020_3daysback_nolikecriterion.pkl', 'rb') as f:
    home_tweets = pickle.load(f)
with open('data/pickled_tweets/away_2020_3daysback_nolikecriterion.pkl', 'rb') as f:
    away_tweets = pickle.load(f)

In [5]:
season = pd.read_csv('data/season_data/2020_all_data.csv', index_col=0)

In [6]:
# count unigrams
def count_unigrams(tweet, corpus):
    words = tweet.split(" ")
    for word in words:
        if word in corpus:
            corpus[word] += 1
        else:
            corpus[word] = 1

In [7]:
train_test_split = int(np.floor(0.75*len(home_tweets)))

In [8]:
train_test_split

192

In [9]:
#Create the home/away corpus
home_corpus = {}
away_corpus = {}
for tweets in home_tweets[:train_test_split]:
    for tw in tweets:
        count_unigrams(tw,home_corpus)

for tweets in away_tweets[:train_test_split]:
    for tw in tweets:
        count_unigrams(tw,away_corpus)

In [10]:
stopwords = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

In [11]:
# remove words with <= 2 characters, stopwords, hashtags
for key in list(home_corpus.keys()):
    if len(key) <= 2:
        del home_corpus[key]
    elif key in stopwords:
        del home_corpus[key]
    elif key[0] == '#':
        del home_corpus[key]

for key in list(away_corpus.keys()):
    if len(key) <= 2:
        del away_corpus[key]
    elif key in stopwords:
        del away_corpus[key]
    elif key[0] == '#':
        del away_corpus[key]

In [12]:
#Turns a list of tweets (for one team, for one game)
#into a vector using the top_grams
from collections import Counter
def vectorize_log(list_of_tweets, corpus):
    num_tweets = len(list_of_tweets)
    lol = [x.split() for x in list_of_tweets]
    with_repeats = [item for sublist in lol for item in sublist]
    counts = dict(Counter(with_repeats))
    to_return = []
    for key in corpus:
        num = np.log(1+counts[key]) if key in counts else 0
        to_return.append(num)
    return to_return

In [13]:
from collections import Counter
def vectorize_normalize(list_of_tweets, corpus):
    num_tweets = len(list_of_tweets)
    lol = [x.split() for x in list_of_tweets]
    with_repeats = [item for sublist in lol for item in sublist]
    counts = dict(Counter(with_repeats))
    to_return = []
    for key in corpus:
        num = counts[key]/num_tweets if key in counts else 0
        to_return.append(num)
    return to_return

In [14]:
# get number of tweets for all home and away teams
n_home_tweets = sum([len(game_tweets) for game_tweets in home_tweets])
n_away_tweets = sum([len(game_tweets) for game_tweets in away_tweets])

# get the unigrams that appear in at least 0.1% of home/away tweets
home_top_grams = [word for word in home_corpus if home_corpus[word] > n_home_tweets*0.001]
away_top_grams = [word for word in away_corpus if away_corpus[word] > n_away_tweets*0.001]

In [15]:
len(home_top_grams)

966

In [16]:
len(away_top_grams)

930

In [17]:
# TRAIN SET
#Create lists of vectors for home, away games
num_h_tweets = [vectorize_log(game, home_top_grams) for game in home_tweets[:train_test_split]]
num_a_tweets = [vectorize_log(game, away_top_grams) for game in away_tweets[:train_test_split]]

#Turn into arrays
home_vecs = np.array(num_h_tweets)
away_vecs = np.array(num_a_tweets)

#Concatenate home, away to form input matrix.
X_train = np.concatenate([home_vecs, away_vecs], axis=1)

# TEST SET
#Create lists of vectors for home, away games
num_h_tweets = [vectorize_log(game, home_top_grams) for game in home_tweets[train_test_split:]]
num_a_tweets = [vectorize_log(game, away_top_grams) for game in away_tweets[train_test_split:]]

#Turn into arrays
home_vecs = np.array(num_h_tweets)
away_vecs = np.array(num_a_tweets)

#Concatenate home, away to form input matrix.
X_test = np.concatenate([home_vecs, away_vecs], axis=1)

In [18]:
Y_train = np.array(season["Home Win"])[:train_test_split]
Y_test = np.array(season["Home Win"])[train_test_split:]

In [19]:
X_train.shape, Y_train.shape

((192, 1896), (192,))

In [20]:
X_test.shape, Y_test.shape

((64, 1896), (64,))

In [21]:
model = Sequential()
model.add(Dense(50, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=50, verbose=False)

In [22]:
model.evaluate(X_test, Y_test)

2/2 [==============================] - 0s 2ms/step - loss: 1.2936 - accuracy: 0.5781


[1.29355788230896, 0.578125]

In [23]:
model = Sequential()
model.add(Dense(128, activation='linear', kernel_regularizer=l1(.001), input_dim=X_train.shape[1]))
model.add(LeakyReLU())
#model.add(Dropout(.5))
model.add(Dense(64, activation='linear',kernel_regularizer=l1(.001)))
model.add(LeakyReLU())
#model.add(Dropout(.5))
model.add(Dense(64, activation='linear',kernel_regularizer=l1(.001)))
model.add(LeakyReLU())
#model.add(Dropout(.5))
model.add(Dense(64, activation='linear',kernel_regularizer=l1(.001)))
model.add(LeakyReLU())
#model.add(Dropout(.5))
model.add(Dense(32, activation='linear',kernel_regularizer=l1(.001)))
model.add(LeakyReLU())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=39, verbose=False)

In [24]:
model.evaluate(X_test, Y_test)

2/2 [==============================] - 0s 2ms/step - loss: 3.2876 - accuracy: 0.6250


[3.2876405715942383, 0.625]